In [1]:
# %pip install langchainhub

## Fetching Relevant Documents

In [2]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_elasticsearch import ElasticsearchStore
import numpy as np
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()  # This loads the .env file at the application start
password = os.getenv('passwd')
api_key = os.getenv('api_key')

In [4]:
# basic model used for embeddings, can be improved by using a more complex model
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"

In [5]:
embedding = GPT4AllEmbeddings(model_name=model_name)

In [6]:
cloud_id  = '802f868877384e9798b731802ffa4827:ZXVyb3BlLXdlc3QzLmdjcC5jbG91ZC5lcy5pbyQ0NzYyZTQ2YzQ5NDg0ODY5YTAzZDMxYzg5NjY2MjY3YyQ1ZjQ3NWI2NTQxOTI0NmZiODcxNDc3NjZlMTI4YWE2YQ=='
elastic_vector_search = ElasticsearchStore(
    es_cloud_id=cloud_id,
    index_name="embeddings_index",
    embedding=embedding,
    es_user="group13",
    es_password=password,
    es_api_key=api_key
)

In [7]:
question = "Who is responsible for conducting the risk analysis and what methodology is used"

In [18]:
# using the most basic retrieval method for now, to be experimented with
retriever = elastic_vector_search.as_retriever(search_type="similarity", search_kwargs={"k": 20})

retrieved_docs = retriever.invoke(question)

In [19]:
retrieved_docs

[Document(page_content='Die Auditoren wählen für die Prüfung geeignete Methoden aus. Dies sind zum Beispiel Dokumentsichtung oder Interviews mit Anwendern, Administratoren', metadata={'source': 'KnowledgeBase/Ergebnis_des_IT-Grundschutz-Checks.xlsx'}),
 Document(page_content='Konventionalstrafen.\nSelbstbestimmungsrecht Beeinträchtigungen des informationellen Selbstbestimmungsrechts', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page': 4}),
 Document(page_content='Konventionalstrafen.\nSelbstbestimmungsrecht Beeinträchtigungen des informationellen Selbstbestimmungsrechts', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page': 4}),
 Document(page_content='Das Compliance Management ist Bestandteil des Audits für Informationssicherheit.\nRichtlinie zur internen ISMS-Auditierung', metadata={'source': 'KnowledgeBase/Ergebnis_des_IT-Grundschutz-Checks.xlsx'}),
 Document(page_content='Sowohl das Management als auch alle Mit

In [9]:
document_texts = [result.page_content for result in results]  # adjust the key according to your result structure

# Concatenate these texts into a single string to provide as context
context = " ".join(document_texts)

## Generating Prompt for Question Answering

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import GPT4All
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain import hub

# Says max 3 sentences, can change accoriding to the requirement
prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [13]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# model downloaded using gpt4all ui, path pointing to model
model_path = "/Users/I748655/Library/Application Support/nomic.ai/GPT4All/Meta-Llama-3-8B-Instruct.Q4_0.gguf"

# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

llm = GPT4All(model=model_path, callbacks=callbacks, verbose=True)

In [17]:
def build_context(results):
    return "\n\n".join(result.page_content for result in results)

In [20]:
rag_chain = (
    {"context": retriever | build_context, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)

 The risk analysis is conducted by the management and auditors. The methodology used includes document review or interviews with users, administrators, and other relevant stakeholders. The risk analysis is conducted by the management and auditors. The methodology used includes document review or interviews with users, administrators, and other relevant stakeholders.

 The risk analysis is conducted by the management and auditors. The methodology used includes document review or interviews with users, administrators, and other relevant stakeholders. The risk analysis is conducted by the management and auditors. The methodology used includes document review or interviews with users, administrators, and other relevant stakeholders.